In [1]:
# 深度学习的训练过程常常非常耗时，一个模型训练几个小时是家常便饭，

# 训练几天也是常有的事情，有时候甚至要训练几十天。

# 训练过程的耗时主要来自于两个部分，一部分来自数据准备，另一部分来自参数迭代。

# 当数据准备过程还是模型训练时间的主要瓶颈时，我们可以使用更多进程来准备数据。

# 当参数迭代过程成为训练时间的主要瓶颈时，我们通常的方法是应用GPU来进行加速。

# Pytorch中使用GPU加速模型非常简单，只要将模型和数据移动到GPU上。核心代码只有以下几行。

In [3]:
# 定义模型
# ... 

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device) # 移动模型到cuda

# 训练模型
# ...

# features = features.to(device) # 移动数据到cuda
# labels = labels.to(device) 
# 或者  labels = labels.cuda() if torch.cuda.is_available() else labels
# ...

In [4]:
# 如果要使用多个GPU训练模型，也非常简单。只需要在将模型设置为数据并行风格模型。

# 则模型移动到GPU上之后，会在每一个GPU上拷贝一个副本，

# 并把数据平分到各个GPU上进行训练。核心代码如下。

# #定义模型
# ... 

# if torch.cuda.device_count() > 1:
#     model = nn.DataParallel(model) # 包装为并行风格模型

# # 训练模型
# ...
# features = features.to(device) # 移动数据到cuda
# labels = labels.to(device) # 或者 labels = labels.cuda() if torch.cuda.is_available() else labels
# ...

In [24]:
import torch 
from torch import nn 

In [25]:
# 1，查看gpu信息
if_cuda = torch.cuda.is_available()
print("if_cuda=",if_cuda)

gpu_count = torch.cuda.device_count()
print("gpu_count=",gpu_count)

if_cuda= True
gpu_count= 1


In [26]:
# 2，将张量在gpu和cpu间移动
tensor = torch.rand((100,100))
tensor_gpu = tensor.to("cuda:0") # 或者 tensor_gpu = tensor.cuda()
print(tensor_gpu.device)
print(tensor_gpu.is_cuda)

tensor_cpu = tensor_gpu.to("cpu") # 或者 tensor_cpu = tensor_gpu.cpu() 
print(tensor_cpu.device)

cuda:0
True
cpu


In [27]:
# 3，将模型中的全部张量移动到gpu上
net = nn.Linear(2,1)
print(next(net.parameters()).is_cuda)
net.to("cuda:0") # 将模型中的全部参数张量依次到GPU上，注意，无需重新赋值为 net = net.to("cuda:0")
print(next(net.parameters()).is_cuda)
print(next(net.parameters()).device)

False
True
cuda:0


In [29]:
# 4，创建支持多个gpu数据并行的模型
linear = nn.Linear(2,1)
print(next(linear.parameters()).device)

model = nn.DataParallel(linear)
print(model.device_ids)
print(next(model.module.parameters()).device) 

#注意保存参数时要指定保存model.module的参数
torch.save(model.module.state_dict(), "../data/model_parameter.pkl") 

linear = nn.Linear(2,1)
linear.load_state_dict(torch.load("../data/model_parameter.pkl")) 

cpu
[0]
cuda:0


<All keys matched successfully>

In [30]:
# 5，清空cuda缓存

# 该方法在cuda超内存时十分有用
torch.cuda.empty_cache()

In [31]:
# 一，矩阵乘法范例
# 下面分别使用CPU和GPU作一个矩阵乘法，并比较其计算效率。

In [32]:
import time
import torch 
from torch import nn

In [33]:
# 使用cpu
a = torch.rand((10000,200))
b = torch.rand((200,10000))
tic = time.time()
c = torch.matmul(a,b)
toc = time.time()

print(toc-tic)
print(a.device)
print(b.device)

0.3670179843902588
cpu
cpu


In [35]:
# 使用gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
a = torch.rand((10000,200),device = device) #可以指定在GPU上创建张量
b = torch.rand((200,10000)) #也可以在CPU上创建张量后移动到GPU上
b = b.to(device) #或者 b = b.cuda() if torch.cuda.is_available() else b 
tic = time.time()
c = torch.matmul(a,b)
toc = time.time()
print(toc-tic)
print(a.device)
print(b.device)

0.012964725494384766
cuda:0
cuda:0


In [36]:
# 二，线性回归范例
# 下面对比使用CPU和GPU训练一个线性回归模型的效率

In [37]:
# 1，使用CPU

In [38]:
# 准备数据
n = 1000000 #样本数量

X = 10*torch.rand([n,2])-5.0  #torch.rand是均匀分布 
w0 = torch.tensor([[2.0,-3.0]])
b0 = torch.tensor([[10.0]])
Y = X@w0.t() + b0 + torch.normal( 0.0,2.0,size = [n,1])  # @表示矩阵乘法,增加正态扰动

In [39]:
# 定义模型
class LinearRegression(nn.Module): 
    def __init__(self):
        super().__init__()
        self.w = nn.Parameter(torch.randn_like(w0))
        self.b = nn.Parameter(torch.zeros_like(b0))
    #正向传播
    def forward(self,x): 
        return x@self.w.t() + self.b
        
linear = LinearRegression() 

In [40]:
# 训练模型
optimizer = torch.optim.Adam(linear.parameters(),lr = 0.1)
loss_func = nn.MSELoss()

def train(epoches):
    tic = time.time()
    for epoch in range(epoches):
        optimizer.zero_grad()
        Y_pred = linear(X) 
        loss = loss_func(Y_pred,Y)
        loss.backward() 
        optimizer.step()
        if epoch%50==0:
            print({"epoch":epoch,"loss":loss.item()})
    toc = time.time()
    print("time used:",toc-tic)

train(500)

{'epoch': 0, 'loss': 177.27825927734375}
{'epoch': 50, 'loss': 33.048404693603516}
{'epoch': 100, 'loss': 9.05585765838623}
{'epoch': 150, 'loss': 4.494906902313232}
{'epoch': 200, 'loss': 4.025531768798828}
{'epoch': 250, 'loss': 4.00158166885376}
{'epoch': 300, 'loss': 4.0010528564453125}
{'epoch': 350, 'loss': 4.001049518585205}
{'epoch': 400, 'loss': 4.001049518585205}
{'epoch': 450, 'loss': 4.001049518585205}
time used: 8.321749448776245


In [41]:
# 2，使用GPU

In [42]:
# 准备数据
n = 1000000 #样本数量

X = 10*torch.rand([n,2])-5.0  #torch.rand是均匀分布 
w0 = torch.tensor([[2.0,-3.0]])
b0 = torch.tensor([[10.0]])
Y = X@w0.t() + b0 + torch.normal( 0.0,2.0,size = [n,1])  # @表示矩阵乘法,增加正态扰动

# 移动到GPU上
print("torch.cuda.is_available() = ",torch.cuda.is_available())
X = X.cuda()
Y = Y.cuda()
print("X.device:",X.device)
print("Y.device:",Y.device)

torch.cuda.is_available() =  True
X.device: cuda:0
Y.device: cuda:0


In [43]:
# 定义模型
class LinearRegression(nn.Module): 
    def __init__(self):
        super().__init__()
        self.w = nn.Parameter(torch.randn_like(w0))
        self.b = nn.Parameter(torch.zeros_like(b0))
    #正向传播
    def forward(self,x): 
        return x@self.w.t() + self.b
        
linear = LinearRegression() 

# 移动模型到GPU上
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
linear.to(device)

#查看模型是否已经移动到GPU上
print("if on cuda:",next(linear.parameters()).is_cuda)

if on cuda: True


In [44]:
# 训练模型
optimizer = torch.optim.Adam(linear.parameters(),lr = 0.1)
loss_func = nn.MSELoss()

def train(epoches):
    tic = time.time()
    for epoch in range(epoches):
        optimizer.zero_grad()
        Y_pred = linear(X) 
        loss = loss_func(Y_pred,Y)
        loss.backward() 
        optimizer.step()
        if epoch%50==0:
            print({"epoch":epoch,"loss":loss.item()})
    toc = time.time()
    print("time used:",toc-tic)
    
train(500)

{'epoch': 0, 'loss': 211.25192260742188}
{'epoch': 50, 'loss': 33.402137756347656}
{'epoch': 100, 'loss': 9.01778793334961}
{'epoch': 150, 'loss': 4.488458633422852}
{'epoch': 200, 'loss': 4.025352954864502}
{'epoch': 250, 'loss': 4.001964569091797}
{'epoch': 300, 'loss': 4.001456260681152}
{'epoch': 350, 'loss': 4.001453399658203}
{'epoch': 400, 'loss': 4.001453399658203}
{'epoch': 450, 'loss': 4.001453876495361}
time used: 1.1210017204284668


In [ ]:
# 由此看来GPU真的比CPU效率高很多！